In [1]:
# import libraries
import os
import sys
import time
import pandas as pd
import numpy as np
from scipy import stats
from scipy.interpolate import CubicSpline
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder

In [2]:
BATCH_SIZE = 256
train_rate = 0.1

In [3]:
# load data without header
data = pd.read_csv('./ISWC21_data_plus_raw/rwhar_3sbjs_data.csv', header=None)
# add header
data.columns = ['subject', 'x', 'y', 'z', 'activity']
data.head()

,subject,x,y,z,activity
0,0,0.378284,10.168175,0.847547,climbing_up
1,0,0.383671,10.172364,0.849942,climbing_up
2,0,0.372298,10.181941,0.859518,climbing_up
3,0,0.342969,10.170568,0.834379,climbing_up
4,0,0.319626,10.159795,0.818817,climbing_up


In [4]:
data.shape

(659260, 5)

In [5]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data['activity'])
data['encoded_activity'] = encoded_labels

In [6]:
data.head()

,subject,x,y,z,activity,encoded_activity
0,0,0.378284,10.168175,0.847547,climbing_up,1
1,0,0.383671,10.172364,0.849942,climbing_up,1
2,0,0.372298,10.181941,0.859518,climbing_up,1
3,0,0.342969,10.170568,0.834379,climbing_up,1
4,0,0.319626,10.159795,0.818817,climbing_up,1


In [7]:
def sliding_window_samples(data, samples_per_window, overlap_ratio):
    """
    Return a sliding window measured in number of samples over a data array along with the mode label for each window.

    :param data: input array, can be numpy or pandas dataframe
    :param samples_per_window: window length as number of samples
    :param overlap_ratio: overlap is meant as percentage and should be an integer value
    :return: tuple of windows, indices, and labels
    """
    windows = []
    indices = []
    labels = []
    curr = 0
    win_len = int(samples_per_window)
    if overlap_ratio is not None:
        overlapping_elements = int((overlap_ratio / 100) * win_len)
        if overlapping_elements >= win_len:
            print('Number of overlapping elements exceeds window size.')
            return
    while curr < len(data) - win_len:
        window = data[curr:curr + win_len]
        windows.append(window.iloc[:, :-2])  # Exclude the last two columns (original and encoded labels)
        indices.append([curr, curr + win_len])
        
        # Extract and compute the mode of the encoded labels for the current window
        window_labels = window['encoded_activity']
        mode_result = mode(window_labels)
        window_label = mode_result[0] if mode_result[0].size > 0 else mode_result
        labels.append(window_label)

        curr += win_len - overlapping_elements

    result_windows = np.array(windows)
    result_indices = np.array(indices)
    result_labels = np.array(labels)
    return result_windows, result_indices, result_labels


In [8]:
sampling_rate = 50
time_window = 2
window_size = sampling_rate * time_window
overlap_ratio = 0

window_data, _, window_label = sliding_window_samples(data, window_size, overlap_ratio)
print(f"shape of window dataset (2 sec with 0% overlap): {window_data.shape}")
print(f"shape of window label (2 sec with 0% overlap): {window_label.shape}")

shape of window dataset (2 sec with 0% overlap): (6592, 100, 4)
shape of window label (2 sec with 0% overlap): (6592,)


In [9]:
#remove the subject column
window_data = window_data[:, :, 1:]

In [10]:
window_data[0].shape

(100, 3)

In [11]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Subset
import numpy as np


window_data = window_data.astype(np.float32)

# Convert to PyTorch tensors
window_data_tensor = torch.from_numpy(window_data)
window_label_tensor = torch.from_numpy(window_label)
#convert labels to long
window_label_tensor = window_label_tensor.long()

# split data into train and test sets
train_size = int(train_rate * len(window_data_tensor))
test_size = len(window_data_tensor) - train_size

# Creating datasets
dataset = TensorDataset(window_data_tensor, window_label_tensor)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Function to extract tensors from Subset
def extract_subset_data(subset, dataset):
    return dataset.tensors[0][subset.indices], dataset.tensors[1][subset.indices]

# Extract data and labels from train and test sets
train_data, train_labels = extract_subset_data(train_dataset, dataset)
test_data, test_labels = extract_subset_data(test_dataset, dataset)

# create train and test TensorDataset
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

# create DataLoader for train and test sets
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
#  print the shape of train_loader and test_loader
print(f"shape of train_loader: {len(train_loader)}")
print(f"shape of test_loader: {len(test_loader)}")

shape of train_loader: 3
shape of test_loader: 24


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNFeatureExtractor(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNFeatureExtractor, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 12, 128)  # Adjust the input features according to your final conv layer output
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [14]:
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu2 = nn.ReLU()
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        
        res = x if self.downsample is None else self.downsample(x)

        # Adjusting the length of the residual to match the output
        if out.size(2) != res.size(2):
            desired_length = out.size(2)
            res = res[:, :, :desired_length]

        return self.relu(out + res)


class TCN(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout=0.2, num_classes=4):
        super(TCN, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size + (dilation_size - 1))]

        self.tcn = nn.Sequential(*layers)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_channels[-1], num_classes)

    def forward(self, x):
        x = self.tcn(x)
        x = F.avg_pool1d(x, x.size(2)).squeeze(2)  # Global Average Pooling
        x = self.dropout(x)
        return self.fc(x)

In [15]:
# create training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.transpose(1, 2)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    return running_loss / len(train_loader)

In [16]:
# create testing function
def test(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    #calculate accuracy
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs = inputs.transpose(1, 2)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            #calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    accuracy = correct / len(test_loader.dataset)
    return running_loss / len(test_loader), accuracy

In [17]:
# create function to train and test model
def train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs):
    train_losses = []
    test_losses = []
    test_accuracies = []
    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        test_loss, test_accuracy = test(model, test_loader, criterion, device)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
        print(f"Epoch: {epoch + 1}/{num_epochs}.. Train Loss: {train_loss:.3f}.. Test Loss: {test_loss:.3f}.. Test Accuracy: {test_accuracy:.3f}")
    return train_losses, test_losses, test_accuracies

In [18]:
# get number of classes
num_classes = len(np.unique(window_label))
print(f"number of classes: {num_classes}")

number of classes: 8


In [19]:
model = CNNFeatureExtractor(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/50.. Train Loss: 1.893.. Test Loss: 1.551.. Test Accuracy: 0.366
Epoch: 2/50.. Train Loss: 1.402.. Test Loss: 1.148.. Test Accuracy: 0.593
Epoch: 3/50.. Train Loss: 1.104.. Test Loss: 0.945.. Test Accuracy: 0.634
Epoch: 4/50.. Train Loss: 0.932.. Test Loss: 0.858.. Test Accuracy: 0.642
Epoch: 5/50.. Train Loss: 0.867.. Test Loss: 0.802.. Test Accuracy: 0.708
Epoch: 6/50.. Train Loss: 0.747.. Test Loss: 0.739.. Test Accuracy: 0.746
Epoch: 7/50.. Train Loss: 0.704.. Test Loss: 0.689.. Test Accuracy: 0.768
Epoch: 8/50.. Train Loss: 0.638.. Test Loss: 0.717.. Test Accuracy: 0.738
Epoch: 9/50.. Train Loss: 0.658.. Test Loss: 0.672.. Test Accuracy: 0.742
Epoch: 10/50.. Train Loss: 0.590.. Test Loss: 0.662.. Test Accuracy: 0.775
Epoch: 11/50.. Train Loss: 0.594.. Test Loss: 0.644.. Test Accuracy: 0.787
Epoch: 12/50.. Train Loss: 0.544.. Test Loss: 0.603.. Test Accuracy: 0.810
Epoch: 13/50.. Train Loss: 0.502.. Test Loss: 0.622.. Test Accuracy: 0.786
Epoch: 14/50.. Train Loss: 0.492..

In [20]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc2.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs)


Epoch: 1/50.. Train Loss: 2.325.. Test Loss: 2.162.. Test Accuracy: 0.231
Epoch: 2/50.. Train Loss: 2.123.. Test Loss: 2.080.. Test Accuracy: 0.163
Epoch: 3/50.. Train Loss: 2.059.. Test Loss: 2.036.. Test Accuracy: 0.160
Epoch: 4/50.. Train Loss: 2.036.. Test Loss: 2.006.. Test Accuracy: 0.161
Epoch: 5/50.. Train Loss: 2.005.. Test Loss: 1.999.. Test Accuracy: 0.158
Epoch: 6/50.. Train Loss: 1.992.. Test Loss: 1.993.. Test Accuracy: 0.145
Epoch: 7/50.. Train Loss: 1.985.. Test Loss: 1.974.. Test Accuracy: 0.141
Epoch: 8/50.. Train Loss: 1.958.. Test Loss: 1.956.. Test Accuracy: 0.149
Epoch: 9/50.. Train Loss: 1.939.. Test Loss: 1.944.. Test Accuracy: 0.171
Epoch: 10/50.. Train Loss: 1.928.. Test Loss: 1.935.. Test Accuracy: 0.156
Epoch: 11/50.. Train Loss: 1.920.. Test Loss: 1.922.. Test Accuracy: 0.155
Epoch: 12/50.. Train Loss: 1.906.. Test Loss: 1.909.. Test Accuracy: 0.155
Epoch: 13/50.. Train Loss: 1.890.. Test Loss: 1.897.. Test Accuracy: 0.157
Epoch: 14/50.. Train Loss: 1.876..

In [21]:
model = CNNFeatureExtractor(num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/cnn_feature_extractor_join_20231218-2006.pt'))

# Freezing layers up to conv3
for name, param in model.named_parameters():
    if 'conv3' in name:
        break
    param.requires_grad = False

# Unfreeze layers from conv3 onwards
unfreeze = False
for name, param in model.named_parameters():
    if 'conv3' in name:
        unfreeze = True
    if unfreeze:
        param.requires_grad = True

# Update the final classification layer
model.fc2 = nn.Linear(in_features=model.fc2.in_features, out_features=num_classes)

# Move the model to the appropriate device
model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer (only update the parameters that require gradients)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# Train and test the model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs)


Epoch: 1/50.. Train Loss: 4.375.. Test Loss: 2.660.. Test Accuracy: 0.119
Epoch: 2/50.. Train Loss: 2.368.. Test Loss: 2.211.. Test Accuracy: 0.186
Epoch: 3/50.. Train Loss: 2.142.. Test Loss: 1.963.. Test Accuracy: 0.256
Epoch: 4/50.. Train Loss: 1.884.. Test Loss: 1.791.. Test Accuracy: 0.341
Epoch: 5/50.. Train Loss: 1.750.. Test Loss: 1.685.. Test Accuracy: 0.289
Epoch: 6/50.. Train Loss: 1.654.. Test Loss: 1.583.. Test Accuracy: 0.323
Epoch: 7/50.. Train Loss: 1.524.. Test Loss: 1.452.. Test Accuracy: 0.334
Epoch: 8/50.. Train Loss: 1.389.. Test Loss: 1.333.. Test Accuracy: 0.421
Epoch: 9/50.. Train Loss: 1.300.. Test Loss: 1.216.. Test Accuracy: 0.566
Epoch: 10/50.. Train Loss: 1.183.. Test Loss: 1.102.. Test Accuracy: 0.615
Epoch: 11/50.. Train Loss: 1.088.. Test Loss: 1.018.. Test Accuracy: 0.704
Epoch: 12/50.. Train Loss: 1.024.. Test Loss: 0.938.. Test Accuracy: 0.681
Epoch: 13/50.. Train Loss: 0.970.. Test Loss: 0.889.. Test Accuracy: 0.720
Epoch: 14/50.. Train Loss: 0.890..

In [22]:
num_inputs = 3  # Assuming 3 input channels (x, y, z axes of the accelerometer)
num_channels = [64, 128, 256]  # Example channel sizes for each layer
kernel_size = 8  # Kernel size for temporal convolutions

model = TCN(num_inputs, num_channels, kernel_size, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)

Epoch: 1/50.. Train Loss: 1.917.. Test Loss: 1.502.. Test Accuracy: 0.432
Epoch: 2/50.. Train Loss: 1.372.. Test Loss: 1.401.. Test Accuracy: 0.506
Epoch: 3/50.. Train Loss: 1.215.. Test Loss: 1.114.. Test Accuracy: 0.534
Epoch: 4/50.. Train Loss: 1.073.. Test Loss: 0.985.. Test Accuracy: 0.593
Epoch: 5/50.. Train Loss: 0.981.. Test Loss: 0.916.. Test Accuracy: 0.588
Epoch: 6/50.. Train Loss: 0.884.. Test Loss: 0.845.. Test Accuracy: 0.664
Epoch: 7/50.. Train Loss: 0.809.. Test Loss: 0.826.. Test Accuracy: 0.682
Epoch: 8/50.. Train Loss: 0.795.. Test Loss: 0.746.. Test Accuracy: 0.730
Epoch: 9/50.. Train Loss: 0.699.. Test Loss: 0.699.. Test Accuracy: 0.745
Epoch: 10/50.. Train Loss: 0.657.. Test Loss: 0.638.. Test Accuracy: 0.792
Epoch: 11/50.. Train Loss: 0.572.. Test Loss: 0.646.. Test Accuracy: 0.774
Epoch: 12/50.. Train Loss: 0.627.. Test Loss: 0.627.. Test Accuracy: 0.787
Epoch: 13/50.. Train Loss: 0.649.. Test Loss: 0.649.. Test Accuracy: 0.781
Epoch: 14/50.. Train Loss: 0.592..

In [23]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs)

Epoch: 1/50.. Train Loss: 3.609.. Test Loss: 2.837.. Test Accuracy: 0.131
Epoch: 2/50.. Train Loss: 2.922.. Test Loss: 2.305.. Test Accuracy: 0.261
Epoch: 3/50.. Train Loss: 2.537.. Test Loss: 1.965.. Test Accuracy: 0.256
Epoch: 4/50.. Train Loss: 2.267.. Test Loss: 1.938.. Test Accuracy: 0.142
Epoch: 5/50.. Train Loss: 2.291.. Test Loss: 1.935.. Test Accuracy: 0.139
Epoch: 6/50.. Train Loss: 2.183.. Test Loss: 1.779.. Test Accuracy: 0.216
Epoch: 7/50.. Train Loss: 2.034.. Test Loss: 1.647.. Test Accuracy: 0.418
Epoch: 8/50.. Train Loss: 1.947.. Test Loss: 1.596.. Test Accuracy: 0.463
Epoch: 9/50.. Train Loss: 1.823.. Test Loss: 1.567.. Test Accuracy: 0.468
Epoch: 10/50.. Train Loss: 1.801.. Test Loss: 1.525.. Test Accuracy: 0.438
Epoch: 11/50.. Train Loss: 1.721.. Test Loss: 1.466.. Test Accuracy: 0.474
Epoch: 12/50.. Train Loss: 1.728.. Test Loss: 1.408.. Test Accuracy: 0.573
Epoch: 13/50.. Train Loss: 1.618.. Test Loss: 1.370.. Test Accuracy: 0.577
Epoch: 14/50.. Train Loss: 1.577..

In [24]:
print(model)

TCN(
  (tcn): Sequential(
    (0): TemporalBlock(
      (conv1): Conv1d(3, 64, kernel_size=(8,), stride=(1,))
      (relu1): ReLU()
      (conv2): Conv1d(64, 64, kernel_size=(8,), stride=(1,))
      (relu2): ReLU()
      (downsample): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
    )
    (1): TemporalBlock(
      (conv1): Conv1d(64, 128, kernel_size=(8,), stride=(1,), dilation=(2,))
      (relu1): ReLU()
      (conv2): Conv1d(128, 128, kernel_size=(8,), stride=(1,), dilation=(2,))
      (relu2): ReLU()
      (downsample): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
    )
    (2): TemporalBlock(
      (conv1): Conv1d(128, 256, kernel_size=(8,), stride=(1,), dilation=(4,))
      (relu1): ReLU()
      (conv2): Conv1d(256, 256, kernel_size=(8,), stride=(1,), dilation=(4,))
      (relu2): ReLU()
      (downsample): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
    )
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Li

In [25]:
model = TCN(num_inputs, num_channels, kernel_size, num_classes=4)

#load pretrained model
model.load_state_dict(torch.load('./models/tcn_join_20231218-2014.pt'))

model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)

# Freezing all layers initially
for param in model.parameters():
    param.requires_grad = False

# Unfreeze layers from the last TemporalBlock's conv2 onwards
num_levels = len(model.tcn)  # Number of TemporalBlocks in your TCN
for i, block in enumerate(model.tcn):
    if i == num_levels - 1:  # Check if it's the last TemporalBlock
        # Unfreeze the conv2 layer and any subsequent layers within this block
        unfreeze = False
        for name, param in block.named_parameters():
            if 'conv2' in name:
                unfreeze = True
            if unfreeze:
                param.requires_grad = True

# Unfreeze the classification layer
for param in model.fc.parameters():
    param.requires_grad = True

model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs)

Epoch: 1/50.. Train Loss: 3.955.. Test Loss: 3.689.. Test Accuracy: 0.340
Epoch: 2/50.. Train Loss: 3.339.. Test Loss: 1.509.. Test Accuracy: 0.515
Epoch: 3/50.. Train Loss: 2.094.. Test Loss: 1.782.. Test Accuracy: 0.257
Epoch: 4/50.. Train Loss: 1.668.. Test Loss: 1.238.. Test Accuracy: 0.621
Epoch: 5/50.. Train Loss: 1.347.. Test Loss: 1.258.. Test Accuracy: 0.480
Epoch: 6/50.. Train Loss: 1.239.. Test Loss: 1.061.. Test Accuracy: 0.693
Epoch: 7/50.. Train Loss: 1.098.. Test Loss: 1.004.. Test Accuracy: 0.671
Epoch: 8/50.. Train Loss: 1.030.. Test Loss: 0.902.. Test Accuracy: 0.758
Epoch: 9/50.. Train Loss: 0.915.. Test Loss: 0.861.. Test Accuracy: 0.719
Epoch: 10/50.. Train Loss: 0.863.. Test Loss: 0.804.. Test Accuracy: 0.769
Epoch: 11/50.. Train Loss: 0.830.. Test Loss: 0.764.. Test Accuracy: 0.776
Epoch: 12/50.. Train Loss: 0.793.. Test Loss: 0.741.. Test Accuracy: 0.784
Epoch: 13/50.. Train Loss: 0.741.. Test Loss: 0.701.. Test Accuracy: 0.778
Epoch: 14/50.. Train Loss: 0.729..